RentCar está desarrollando una política de reemplazo para su flotilla de automóviles en
un horizonte de planeaci ́on de 4 años. Al inicio de cada a ̃no, un automóvil se reemplaza o
se conserva en operación durante un año más. Un automóvil debe estar en servicio de 1 a 3
años. La siguiente tabla proporciona el costo de reemplazo como una función del año en que
se adquiere un automóvil y los años en operación.

In [25]:
import pandas as pd
import numpy as np

In [35]:
costs = {0: [4000, 5400, 9800], 1: [4300, 6200, 8700], 2: [4800, 71000, np.nan], 3: [4900, np.nan, np.nan]}
columns = [1, 2, 3]

actions = [
    (start_year, start_year + columns[end_year], cost)
    for start_year, cost_list in costs.items()
    for end_year, cost in enumerate(cost_list)
    if not pd.isnull(cost)
]

pd.DataFrame(actions, columns=['start_year', 'end_year', 'cost'])


,start_year,end_year,cost
0,0,1,4000
1,0,2,5400
2,0,3,9800
3,1,2,4300
4,1,3,6200
5,1,4,8700
6,2,3,4800
7,2,4,71000
8,3,4,4900


In [27]:
n_ants = 10  # Número de hormigas
n_iterations = 100  # Número de iteraciones
alpha = 1.0  # Importancia de las feromonas
beta = 1.0  # Importancia de la heurística
evaporation_rate = 0.5  # Tasa de evaporación de las feromonas
pheromone_init = 0.1  # Valor inicial de feromonas